In [1]:
# setup python path
import sys
sys.path.append('/home/r2d9/workshop')

In [2]:
import os

import pandas as pd

from workshop.settings import DATA_PATH

# Init

In [6]:
MIN_PPL = 125
MAX_PPL = 300

# Baseline : filling day in order of wishes
we parse families in the order of the given data and set them to the first of their choice that has available places.

In [20]:
data = pd.read_csv(os.path.join(DATA_PATH, "family_data.csv"), index_col="family_id")

constraints = pd.Series(0, index=[i for i in range(1, 101)], name="constraint")

for family_id, family_data in data.iterrows():
    family_size = family_data["n_people"]
    family_choices = family_data.drop("n_people")
    for choice in family_choices:
        if constraints.loc[choice] + family_size <= MAX_PPL:
            constraints.loc[choice] += family_size
            data.loc[family_id, "assigned_day"] = choice
            break

In [21]:
constraint_checker = data.groupby("assigned_day")["n_people"].sum()
constraint_checker.min() >= MIN_PPL, constraint_checker.max() <= MAX_PPL, "Sounds good doesn't work - D.J. Trump"

(False, True, "Sounds good doesn't work - D.J. Trump")

# Baseline v2 : taking each day and filling
we parse days and fill places (at least 125 then stop) by looking who is the most willing to go on that day

In [25]:
data = pd.read_csv(os.path.join(DATA_PATH, "family_data.csv"), index_col="family_id")

n_choices = ["choice_{}".format(i) for i in range(10)]  # for convenience when parsing data columns
assigned = list()  # holds people already assigned

for day in [i for i in range(1, 101)]:
    
    day_people_number = 0
    
    for n_choice in n_choices:
        
        if day_people_number < MIN_PPL:
            
            curr_most_interested = data[data[n_choice] == day]
            
            for family_id, family_data in curr_most_interested.iterrows():
                if day_people_number < MIN_PPL and family_id not in assigned:
                    day_people_number += family_data["n_people"]
                    data.loc[family_id, "assigned_day"] = day
                    assigned.append(family_id)
                else:
                    break
        else:
            break

In [29]:
constraint_checker = data.groupby("assigned_day")["n_people"].sum()
constraint_checker.min() >= MIN_PPL, constraint_checker.max() <= MAX_PPL

(False, True)

# Baseline v3